In [1]:
pwd


'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4
!pip install tensorflow==1.14.0

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [3]:
import numpy as np
import tensorflow 
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers 

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D 
from keras.preprocessing.image import ImageDataGenerator


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-pack

In [4]:
tensorflow.__version__

'1.14.0'

In [5]:
tensorflow.keras.__version__

'2.2.4-tf'

# Image Data Agumentation

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [13]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_7d01c3a53d364926a1681543db957042 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_7d01c3a53d364926a1681543db957042 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_7d01c3a53d364926a1681543db957042 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='JgJzfbbo7B9qxmS1RCP6a2RGNgKAMWVIv1X_vnNos2XO',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_7d01c3a53d364926a1681543db957042)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_10 = client_7d01c3a53d364926a1681543db957042.get_object(Bucket='imageclassification-donotdelete-pr-r0zoamtbaszqeg', Key='dataset.zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_10, "__iter__"): streaming_body_10.__iter__ = types.MethodType( __iter__, streaming_body_10 ) 


In [14]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_10.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [15]:
pwd

'/home/wsuser/work'

In [16]:
import os
filenames = os.listdir('/home/wsuser/work/dataset/train_set')

# Loading our data and performing data agumentation

In [17]:

x_train = train_datagen.flow_from_directory("/home/wsuser/work/dataset/train_set",target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode="categorical")

x_test = test_datagen.flow_from_directory("/home/wsuser/work/dataset/test_set",target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode="categorical")

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [18]:
print(x_train.class_indices)

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [19]:
print(x_test.class_indices)

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [20]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

# creating the model


In [21]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

# compiling the model

In [23]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# fitting the model

In [24]:
 classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=10, validation_data=x_test,validation_steps = len(x_test))

Epoch 1/10
149/149 [==============================] - 32s 218ms/step - loss: 1.2729 - acc: 0.4124 - val_loss: 1.2354 - val_acc: 0.3636
Epoch 2/10
149/149 [==============================] - 30s 202ms/step - loss: 0.9520 - acc: 0.5849 - val_loss: 0.9171 - val_acc: 0.6616
Epoch 3/10
149/149 [==============================] - 29s 194ms/step - loss: 0.7683 - acc: 0.6604 - val_loss: 0.6776 - val_acc: 0.7475
Epoch 4/10
149/149 [==============================] - 29s 197ms/step - loss: 0.6508 - acc: 0.7466 - val_loss: 0.7413 - val_acc: 0.7374
Epoch 5/10
149/149 [==============================] - 29s 194ms/step - loss: 0.6546 - acc: 0.7345 - val_loss: 0.7710 - val_acc: 0.6768
Epoch 6/10
149/149 [==============================] - 29s 195ms/step - loss: 0.6056 - acc: 0.7453 - val_loss: 0.6295 - val_acc: 0.7424
Epoch 7/10
149/149 [==============================] - 30s 199ms/step - loss: 0.5392 - acc: 0.7817 - val_loss: 0.7014 - val_acc: 0.7323
Epoch 8/10
149/149 [==============================] - 2

# saving our model

In [25]:
classifier.save('disaster.h5')

In [26]:
!tar -zcvf image-classification-model_new.tgz disaster.h5

disaster.h5


In [27]:
ls -1

dataset/
disaster.h5
image-classification-model_new.tgz
my_model.tar.gz


In [28]:
!pip install watson-machine-learning-client --upgrade

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Requirement already up-to-date: watson-machine-learning-client in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (1.0.391)


In [29]:
from ibm_watson_machine_learning import APIClient
wml_credentials={"url":"https://us-south.ml.cloud.ibm.com",
                "apikey":"jgOtjSk8dNpif80Z8fXmgYY6Yk4C_qs4xvKoFedjwRaH"}
client = APIClient(wml_credentials)

In [30]:
client = APIClient(wml_credentials)

In [31]:
def guid_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [32]:
space_uid=guid_space_name(client,'imageclassification')
print("space UID = "+space_uid)

space UID = 747c36a0-5ec5-4348-9a7b-ec6bb50b63af


In [33]:
client.set.default_space(space_uid)

'SUCCESS'

In [34]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029a-cc97-56da-b8e0-39c3880dbbe7  base
tensorflow_2.1-py3.6           1eb25b8

In [35]:
software_space_uid=client.software_specifications.get_uid_by_name("default_py3.7")
software_space_uid

'e4429883-c883-42b6-87a8-f419d64088cd'

In [36]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"AI_based_Natural_disaster_analysis",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid}
                                            )
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [37]:
model_id

'5337af53-bbf5-49b8-aaf1-9335df39c5a4'

In [43]:
client.repository.download(model_id,'model.tar.gz')

Successfully saved model content to file: 'model.tar.gz'


'/home/wsuser/work/model.tar.gz'

In [44]:
import tensorflow as tf
from tensorflow import keras
import h5py

# Initialization

loaded_model = keras.models.load_model('disaster.h5')

In [50]:
from keras.preprocessing import image

In [51]:

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_11 = client_7d01c3a53d364926a1681543db957042.get_object(Bucket='imageclassification-donotdelete-pr-r0zoamtbaszqeg', Key='cyclone.jpg')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_11, "__iter__"): streaming_body_11.__iter__ = types.MethodType( __iter__, streaming_body_11 ) 
img = image.load_img(streaming_body_11,target_size=(64,64))

In [52]:
import numpy as np
x=image.img_to_array(img)
x =np.expand_dims(x,axis=0)

# predicting

In [53]:
pred=classifier.predict_classes(x)

In [54]:
pred[0]

0

In [55]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[pred[0]])
result

'Cyclone'